In [ ]:
#importing lbraries
import pandas as pd
import os
import re

In [ ]:
##IMPORTANT PART TO KEEP CARE OF WHILE RUNNING CODE
# 1. Specify the input directory where original files are present
# 2. Specify the output directory where parsed files are to be kept
# 3. Code is written to parse Manangement Discussion under Item 7 in SEC Fillings but can be modified for any parsing
# 4. Modification can be easily done by using extract_text_between() function
# 5. View the log file to view for the files where no parsing happened

In [ ]:
import re  # Import the regular expression module

def extract_text_between(text, start_word, end_word):
    # Function to extract text between start_word and end_word in the provided text

    if end_word is None:
        # If end_word is not specified, extract text from start_word until the end of the text
        pattern = re.escape(start_word) + r'(.*)'  # Create a regex pattern to match start_word followed by any characters
    else:
        # If end_word is specified, extract text between start_word and end_word
        pattern = re.escape(start_word) + r'(.*?)' + re.escape(end_word)  # Create a regex pattern to match start_word, any characters, and end_word

    # Find all matches of the pattern in the input text
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)  # Use re.findall to find all matches, ignoring case and matching newline characters

    # Return the list of matches found
    return matches


In [ ]:
import os  # Import the os module for file operations
import re  # Import the regular expression module

# Directory paths
input_directory = ''  # Specify the input directory path where the text files are located
output_directory = ''  # Specify the output directory path where the extracted text files will be saved
log = []  # Initialize an empty list to store the filenames for which Item 7 or Item 7A is not found

# Regular expression patterns
accession_number_regex = r'(?i)ACCESSION NUMBER:(.*?)\n'  # Define a regex pattern to match the accession number

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate over each file in the input directory
for filename in os.listdir(input_directory):
    input_file_path = os.path.join(input_directory, filename)  # Create the full path of the input file

    # Process only the text files
    if filename.endswith('.txt'):
        with open(input_file_path, 'r') as file:
            text = file.read()  # Read the contents of the text file

            # Try different patterns to extract the desired text
            matches = extract_text_between(text, 'ITEM 7. Management', 'ITEM 7A. QUANTITATIVE')
            if not matches:
                matches = extract_text_between(text, 'ITEM 7', 'ITEM 9')
            if not matches:
                matches = extract_text_between(text, 'Management', 'QUANTITATIVE')
            if not matches:
                matches = extract_text_between(text, 'Management Discussion')

            accession_number_matches = re.findall(accession_number_regex, text)  # Find all matches of the accession number pattern in the text

            if matches and accession_number_matches:
                accession_number = accession_number_matches[0].strip()  # Get the first match of the accession number
                output_file_path = os.path.join(output_directory, f'{accession_number}_{filename}_Item7.txt')  # Create the full path of the output file

                with open(output_file_path, 'w') as output_file:
                    output_file.write(f'Accession Number: {accession_number}\n\n')
                    for match in matches:
                        output_file.write(match.strip() + '\n')  # Write the extracted text to the output file

            else:
                log.append(filename)  # Add the filename to the log list if Item 7 or Item 7A is not found

print("Text extraction and saving complete.")
